In [1]:
import torch
from pyannote.database import get_protocol, FileFinder

# Load embedding model
emb = torch.hub.load('pyannote/pyannote-audio', 'emb')
print(f'Embedding has dimension {emb.dimension:d}.')

Downloading: "https://github.com/pyannote/pyannote-audio/archive/master.zip" to /home/jovyan/.cache/torch/hub/master.zip


  0%|          | 0.00/901 [00:00<?, ?B/s]

  0%|          | 0.00/16.3M [00:00<?, ?B/s]

/opt/conda/lib/python3.9/site-packages/pyannote/audio/embedding/approaches/arcface_loss.py:170: FutureWarning: The 's' parameter is deprecated in favor of 'scale', and will be removed in a future release
  warnings.warn(msg, FutureWarning)


Embedding has dimension 512.


#### Load audio files protocol

In [2]:
preprocessors = {'audio': FileFinder()}
protocol = get_protocol('VOXCON.SpeakerDiarization.Sample', preprocessors=preprocessors)

#### Get single file for testing

In [ ]:
from pyannote.audio.features.utils import get_audio_duration

train_file = next(protocol.train())
duration = get_audio_duration(train_file)

#### Create SlidingWindow for embedding extraction from utterances

In [ ]:
from pyannote.core import SlidingWindow, Segment

sw = SlidingWindow(duration=0.025, step=0.01, start=0.0, end=duration)

# Test SlidingWindow
# for chunk in sw(Segment(3, 7.5)):
#     print(tuple(chunk))

### D-Vector Create

In [ ]:
def concat_segs(times, segs):
    #Concatenate continuous voiced segments
    concat_seg = []
    seg_concat = segs[0]
    for i in range(0, len(times)-1):
        if times[i][1] == times[i+1][0]:
            seg_concat = np.concatenate((seg_concat, segs[i+1]))
        else:
            concat_seg.append(seg_concat)
            seg_concat = segs[i+1]
    else:
        concat_seg.append(seg_concat)
    return concat_seg

def align_embeddings(embeddings):
    partitions = []
    start = 0
    end = 0
    j = 1
    for i, embedding in enumerate(embeddings):
        if (i*.12)+.24 < j*.401:
            end = end + 1
        else:
            partitions.append((start,end))
            start = end
            end = end + 1
            j += 1
    else:
        partitions.append((start,end))
    avg_embeddings = np.zeros((len(partitions),512))
    for i, partition in enumerate(partitions):
        avg_embeddings[i] = np.average(embeddings[partition[0]:partition[1]],axis=0) 
    return avg_embeddings

In [5]:
import glob
import librosa
import numpy as np
import os
import torch

from hparam import hparam as hp
from VAD_segments import VAD_chunk

In [ ]:
# Split test and train set from the same folder

# audio_path = glob.glob(os.path.dirname(hp.unprocessed_data))  

# total_speaker_num = len(audio_path)
# train_speaker_num= (total_speaker_num//10)*9            # split total data 90% train and 10% test

# audio_path

In [6]:
next(protocol.train())['audio'].as_posix()

'/home/jovyan/work/voxsrc21-dia/data/voxconverse/sample/abjxc.wav'

In [ ]:
train_sequence = []
train_cluster_id = []
label = 0
count = 0
train_saved = False
listLength = len(list(protocol.train()))
for i, file in enumerate(protocol.train()):
    filePath = file['audio'].as_posix()
    times, segs = VAD_chunk(2, filePath)
    if segs == []:
        print('No voice activity detected')
        continue
    concat_seg = concat_segs(times, segs)
#     STFT_frames = get_STFTs(concat_seg)
#     STFT_frames = np.stack(STFT_frames, axis=2)
#     STFT_frames = torch.tensor(np.transpose(STFT_frames, axes=(2,1,0)))
#     print(STFT_frames)
    embeddings = emb(file)
    aligned_embeddings = align_embeddings(embeddings.data)
    train_sequence.append(aligned_embeddings)
    for embedding in aligned_embeddings:
        train_cluster_id.append(str(label))
    count = count + 1
    if count % 100 == 0:
        print('Processed {0}/{1} files'.format(count, listLength))
    label = label + 1
    
#     if not train_saved and i > train_speaker_num:
#         train_sequence = np.concatenate(train_sequence,axis=0)
#         train_cluster_id = np.asarray(train_cluster_id)
#         np.save('train_sequence',train_sequence)
#         np.save('train_cluster_id',train_cluster_id)
#         train_saved = True
#         train_sequence = []
#         train_cluster_id = []
        
train_sequence = np.concatenate(train_sequence,axis=0)
train_cluster_id = np.asarray(train_cluster_id)
np.save('test_sequence',train_sequence)
np.save('test_cluster_id',train_cluster_id)
